# Exercise 4: Distributed Execution Concepts

## Learning Objectives
- Understand partitioning and data distribution
- Analyze shuffle operations and their costs
- Read and interpret the Spark UI (stages, tasks, DAG)
- Optimize jobs by understanding execution plans

---

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time

spark = SparkSession.builder \
    .appName("Distributed Execution Lab") \
    .master("yarn") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.instances", "2") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print(f"Application: {spark.sparkContext.applicationId}")
print("Open Spark UI at http://localhost:4040")

## Part 1: Understanding Partitions

In [ ]:
# Load data and check partitions
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs:///user/student/data/transactions.csv")

print(f"Default partitions: {df.rdd.getNumPartitions()}")

In [ ]:
# Repartition to control parallelism
df_4_parts = df.repartition(4)
df_8_parts = df.repartition(8)

print(f"4 partitions: {df_4_parts.rdd.getNumPartitions()}")
print(f"8 partitions: {df_8_parts.rdd.getNumPartitions()}")

In [ ]:
# See data distribution across partitions
def show_partition_sizes(df, name):
    sizes = df.rdd.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()
    print(f"{name}: {sizes} (total: {sum(sizes)})")

show_partition_sizes(df_4_parts, "4 partitions")
show_partition_sizes(df_8_parts, "8 partitions")

## Part 2: Analyzing Stages and Shuffles

In [ ]:
# Simple aggregation (causes shuffle)
start = time.time()

result1 = df.groupBy("store_region") \
    .agg(count("*").alias("count"), sum("total_amount").alias("total")) \
    .collect()

print(f"Aggregation took: {time.time() - start:.2f} seconds")
print("Check Spark UI -> Stages tab")

In [ ]:
# View execution plan
df.groupBy("store_region") \
    .agg(count("*").alias("count")) \
    .explain(True)

### 🔍 Checkpoint Question 4
Look at the Spark UI Stages tab:
- How many stages were created for the aggregation?
- Find the "Shuffle Read" and "Shuffle Write" metrics. What do they tell you?

**Your Answer:**

## Part 3: Join Strategies

In [ ]:
# Load products (small table)
products = spark.read.json("hdfs:///user/student/data/catalog.json")
print(f"Products size: {products.count()} rows")

In [ ]:
# Regular join (Spark will choose broadcast for small table)
joined = df.join(products, "product_id")
joined.explain(True)

In [ ]:
# Force broadcast hint
from pyspark.sql.functions import broadcast

joined_broadcast = df.join(broadcast(products), "product_id")
joined_broadcast.explain()

In [ ]:
# Execute and measure
start = time.time()
count = joined_broadcast.count()
print(f"Join result: {count} rows in {time.time() - start:.2f} seconds")

## Part 4: DAG Visualization

Complex query with multiple stages

In [ ]:
# Complex query: Join, filter, aggregate, sort
complex_result = df.join(products, "product_id") \
    .filter(col("is_online") == True) \
    .groupBy("category", "store_region") \
    .agg(
        count("*").alias("transactions"),
        sum("total_amount").alias("revenue")
    ) \
    .orderBy(desc("revenue"))

# View the DAG
complex_result.explain(True)

In [ ]:
# Execute
complex_result.show(10)
print("Check Spark UI -> SQL tab for visual DAG")

## Part 5: Caching for Performance

In [ ]:
# Cache frequently used data
df.cache()
df.count()  # Materialize cache

print("Data cached! Check Storage tab in Spark UI")

In [ ]:
# Multiple queries on cached data
start = time.time()
for region in ['North', 'South', 'East', 'West', 'Central']:
    count = df.filter(col("store_region") == region).count()
    print(f"{region}: {count}")
print(f"Total time: {time.time() - start:.2f} seconds")

In [ ]:
# Unpersist to free memory
df.unpersist()
spark.stop()